### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 使用 colab 環境的同學請執行以下程式碼
# %tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
# import tensorflow as tf
# print(tf.__version__)

# import os
# from google.colab import drive 
# drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
# %cd 'gdrive/My Drive'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

In [3]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [4]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [5]:
imgs_train = imgs_train[..., np.newaxis]
imgs_train.shape

(2140, 96, 96, 1)

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import losses

Using TensorFlow backend.


In [7]:
# 定義人臉關鍵點檢測網路
model = Sequential()

# 定義神經網路的輸入, hidden layer 以及輸出

model.add(Conv2D(16, (3, 3), input_shape=(96, 96, 1), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(30))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 94, 94, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxP

In [8]:
# 配置 loss funtion 和 optimizer
model.compile(loss=losses.mae, optimizer='adam')

history = model.fit(imgs_train, points_train, validation_split=0.2, epochs=100, batch_size=16)

Instructions for updating:
Use tf.cast instead.
Train on 1712 samples, validate on 428 samples
Epoch 1/100
1712/1712 [==============================] - 3s 2ms/step - loss: 0.0415 - val_loss: 0.0327
Epoch 2/100
1712/1712 [==============================] - 1s 449us/step - loss: 0.0265 - val_loss: 0.0326
Epoch 3/100
1712/1712 [==============================] - 1s 449us/step - loss: 0.0248 - val_loss: 0.0328
Epoch 4/100
1712/1712 [==============================] - 1s 449us/step - loss: 0.0236 - val_loss: 0.0322
Epoch 5/100
1712/1712 [==============================] - 1s 467us/step - loss: 0.0227 - val_loss: 0.0324
Epoch 6/100
1712/1712 [==============================] - 1s 454us/step - loss: 0.0219 - val_loss: 0.0317
Epoch 7/100
1712/1712 [==============================] - ETA: 0s - loss: 0.020 - 1s 452us/step - loss: 0.0206 - val_loss: 0.0303
Epoch 8/100
1712/1712 [==============================] - 1s 450us/step - loss: 0.0199 - val_loss: 0.0296
Epoch 9/100
1712/1712 [====================

Epoch 75/100
1712/1712 [==============================] - 1s 444us/step - loss: 0.0131 - val_loss: 0.0232
Epoch 76/100
1712/1712 [==============================] - 1s 436us/step - loss: 0.0132 - val_loss: 0.0230
Epoch 77/100
1712/1712 [==============================] - 1s 440us/step - loss: 0.0133 - val_loss: 0.0232
Epoch 78/100
1712/1712 [==============================] - 1s 456us/step - loss: 0.0131 - val_loss: 0.0230
Epoch 79/100
1712/1712 [==============================] - 1s 441us/step - loss: 0.0131 - val_loss: 0.0243
Epoch 80/100
1712/1712 [==============================] - 1s 439us/step - loss: 0.0132 - val_loss: 0.0240
Epoch 81/100
1712/1712 [==============================] - 1s 442us/step - loss: 0.0131 - val_loss: 0.0228
Epoch 82/100
1712/1712 [==============================] - 1s 439us/step - loss: 0.0131 - val_loss: 0.0226
Epoch 83/100
1712/1712 [==============================] - 1s 452us/step - loss: 0.0131 - val_loss: 0.0232
Epoch 84/100
1712/1712 [======================